In [1]:
import warnings
warnings.filterwarnings('ignore')
from numpy import *
import numpy as np
import pandas as pd
import sys, os
import torch
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from numpy.random import randint
from IMOVNN import IMOVNN
import torch.nn.functional as F

In [2]:
# read data
meta_data=pd.read_csv('meta.csv',index_col=0)
metatranscriptomics_non_species=pd.read_csv('metatranscriptomics_non_species.csv',index_col=0).fillna(0)
metagenomics=pd.read_csv('metagenomics.csv',index_col=0)
data_class={'nonIBD':0,'CD':1,'UC':1}
meta_data['diagnosis']=meta_data['diagnosis'].map(data_class)
#对宏基因组数据预处理
def log(base,x):
    return np.log(x)/np.log(base)
metagenomics=log(2,2*metagenomics+0.00001)

x1=metagenomics
x2=metatranscriptomics_non_species
print('输入各组学数据表：',x1.shape,x2.shape,meta_data.shape)

#重构数据表和标签表
X1=pd.concat([x1,x2],axis=1).iloc[:,:x1.shape[1]]
X2=pd.concat([x1,x2],axis=1).iloc[:,x1.shape[1]:x1.shape[1]+x2.shape[1]]
index_new=X1.index
print(index_new)
                                        
meta_data=meta_data.reindex(index_new)
X1=array(X1.reindex(index_new))
X2=array(X2.reindex(index_new))
print('集合所有样本重构数据表：',X1.shape,X2.shape,meta_data.shape)
X_set=[X1,X2]
M        = len(X_set)
Mask     = np.ones([np.shape(X_set[0])[0], M])
for m_idx in range(M):
    Mask[np.isnan(X_set[m_idx]).all(axis=1), m_idx] = 0
#print(Mask)
#print(X_set)
Y = meta_data['diagnosis'].tolist()
Y = np.array(Y)
Y_onehot = np.zeros((Y.shape[0], 2))
Y_onehot[np.arange(Y.shape[0]), Y] = 1
print('标签转换为one-hot编码：', Y_onehot.shape, np.sum(Y_onehot[:, 0]))

输入各组学数据表： (817, 578) (817, 421) (817, 1)
Index(['MSM5LLFM_P', 'HSM6XRRB', 'HSM67VD2', 'HSM7J4IP', 'MSM79H9G',
       'HSMA33RX', 'HSM7J4PM', 'HSM7J4HM', 'MSM79HBB', 'ESM5GEXY',
       ...
       'HSM7J4NY', 'MSM79H9Y', 'PSM7J1AU', 'HSM7J4LN', 'MSM5LLIQ_P',
       'CSMAG78W', 'CSM79HIN', 'HSMA33O3', 'MSM5LLFO_P', 'HSM6XRR5'],
      dtype='object', name='External ID', length=817)
集合所有样本重构数据表： (817, 578) (817, 421) (817, 1)
标签转换为one-hot编码： (817, 2) 200.0


In [3]:
print(sum(Mask))

1634.0


In [4]:
# 指定保存模型的文件夹路径
folder_path = 'model'
# 创建文件夹
os.makedirs(folder_path, exist_ok=True)

cv = StratifiedKFold(n_splits=5,shuffle = True,random_state = 50)
acc_cv = []
f1_cv = []
auc_cv = []
for i,(train_index, test_index) in enumerate(cv.split(X_set[0],np.argmax(Y_onehot,axis=1))):
    select_num = 130
    fs_mg = open(str(select_num)+'_mgmt_mg.txt',mode='a')
    fs_mt = open(str(select_num)+'_mgmt_mt.txt',mode='a')
    fs_prob = open(str(select_num)+'_mgmt_prob.txt',mode='a')
    acc = open(str(select_num)+'_mgmt_acc.txt',mode='a')
    f1 = open(str(select_num)+'_mgmt_f1.txt',mode='a')
    auc = open(str(select_num)+'_mgmt_auc.txt',mode='a')
    tr_X_set, te_X_set, va_X_set = {}, {}, {}
    for m in range(len(X_set)):
        tr_X_set[m],tr_Y_onehot,tr_M = X_set[m][train_index],Y_onehot[train_index],Mask[train_index]
        te_X_set[m],te_Y_onehot,te_M = X_set[m][test_index],Y_onehot[test_index],Mask[test_index]
        #归一化
        #print(te_Y_onehot)
    def Normalize(data):
        """
        :param data:Input data
        :return:normalized data
        """
        
        m = np.mean(data)
        mx = np.max(data)
        mn = np.min(data)
        return (data - m) / (mx - mn)
    
    for m in range(M):
        tr_X_set[m] = Normalize(tr_X_set[m])
        te_X_set[m] = Normalize(te_X_set[m])
    
    x_dim_set    = [tr_X_set[m].shape[1] for m in range(len(tr_X_set))]
    y_dim        = np.shape(tr_Y_onehot)[1]
    
    input_dims = {
        'x_dim_set': x_dim_set,
        'y_dim': y_dim,
        'z_dim': 50,
        #'steps_per_batch': steps_per_batch
    }
    network_settings = {
        'dim_enc': 100,
        'num_layers_enc': 3,   #view-specific
        'dim_specificpre': 100,
        'num_layers_specificpre': 2,  #multi-view
        'dim_joint_pre': 100,
        'num_layers_jointpre': 2,
        'dropout': 0.1,
        'F': select_num,
        'start_temp': 10, 
        'min_temp': 0.1,
        'ITERATION': 10000
    }
    
    model = IMOVNN(input_dims, network_settings, cuda = True)
    model = model.cuda()
    model_save, maxacc, maxf1, maxauc = model.train_model(tr_X_set, tr_Y_onehot, tr_M, te_X_set, te_Y_onehot, te_M, alpha = 1.0, beta = 0.001, l_rate = 0.005)#here test data as val data
    model_path = os.path.join(folder_path, 'model_'+str(i)+'.pth')
    torch.save(model_save, model_path)
    acc_cv.append(maxacc)
    f1_cv.append(maxf1)
    auc_cv.append(maxauc)
    fs_a = model.feature_selective_layer[0].logits.data.argmax(axis = 1)
    fs_b = model.feature_selective_layer[1].logits.data.argmax(axis = 1)
    print(fs_a)
    print(fs_b)
    fs_mg.writelines(str(fs_a)+'\n')
    fs_mt.writelines(str(fs_b)+'\n')
    fs_prob.writelines(str(i)+'\n')
    fs_prob.writelines(str(torch.mean(torch.max(F.softmax(model.feature_selective_layer[0].logits.data, dim = -1), dim = -1)[0]))+'\n')
    fs_prob.writelines(str(torch.mean(torch.max(F.softmax(model.feature_selective_layer[1].logits.data, dim = -1), dim = -1)[0]))+'\n')
    acc.writelines(str(maxacc)+'\n')
    f1.writelines(str(maxf1)+'\n')
    auc.writelines(str(maxauc)+'\n')
    fs_mg.close()
    fs_mt.close()
    fs_prob.close()
    acc.close()
    f1.close()
    auc.close()
print('acc',acc_cv)
print('average maxacc',sum(acc_cv)/5)
print('f1',f1_cv)
print('average maxf1',sum(f1_cv)/5)
print('auc',auc_cv)
print('average maxauc',sum(auc_cv)/5)
acc = open(str(select_num)+'_mgmt_acc.txt',mode='a')
acc.writelines('average:'+str(sum(acc_cv)/5)+'\n')
acc.close()
f1 = open(str(select_num)+'_mgmt_f1.txt',mode='a')
f1.writelines('average:'+str(sum(f1_cv)/5)+'\n')
f1.close()
auc = open(str(select_num)+'_mgmt_auc.txt',mode='a')
auc.writelines('average:'+str(sum(auc_cv)/5)+'\n')
auc.close()

temp: tensor([9.5500], device='cuda:0')
mean max of probabilities 0: tensor(0.0505, device='cuda:0')
mean max of probabilities 1: tensor(0.0696, device='cuda:0')
Train ACC: 0.9433
Test ACC: 0.8293
Test F1: 0.8862
Test AUC: 0.9163
temp: tensor([9.1160], device='cuda:0')
mean max of probabilities 0: tensor(0.1724, device='cuda:0')
mean max of probabilities 1: tensor(0.3385, device='cuda:0')
Train ACC: 0.9985
Test ACC: 0.8841
Test F1: 0.9237
Test AUC: 0.9280
temp: tensor([8.7017], device='cuda:0')
mean max of probabilities 0: tensor(0.1867, device='cuda:0')
mean max of probabilities 1: tensor(0.5719, device='cuda:0')
Train ACC: 1.0000
Test ACC: 0.8780
Test F1: 0.9206
Test AUC: 0.9241
temp: tensor([8.3062], device='cuda:0')
mean max of probabilities 0: tensor(0.1878, device='cuda:0')
mean max of probabilities 1: tensor(0.8608, device='cuda:0')
Train ACC: 1.0000
Test ACC: 0.8902
Test F1: 0.9280
Test AUC: 0.9266
00500: TRAIN| LT=0.326 LP=0.000 LKL=9.367 LPS=0.313 LKLS=3.121 | 
temp: tensor([